In [11]:
import random 
import numpy as np 
import sys
import os
sys.path.insert(0, 'src')
import matplotlib.pyplot as plt
import importlib
import platform
import subprocess
import pickle
from os.path import exists
import csv


# stl stuff for creating stl formulae and calculating robustness
import torch

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu" 
device = torch.device(dev)

import stlcg
# import stlviz as viz
from stlcg.stlcg import Expression
from stlcg.utils import print_learning_progress


# for clustering, dimension reduction, feature selection
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from random import randint
import pandas as pd
import pickle
import time
import timeit
import scipy
from mpl_toolkits.mplot3d import Axes3D
import regex
from scipy.io import savemat
print(os.getcwd())
%matplotlib inline

# #concurrent stuff (didn't work due to morpheus structure)
# from concurrent.futures import ProcessPoolExecutor
# from multiprocessing import Pool
# import ray
# from dask.distributed import Client
# from joblib import Parallel, delayed
# keras stuff for image processing 
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
from keras.models import Model
import tensorflow as tf
print(tf.__version__)
# print(torch.cuda.get_device_name(0))

/storage/coda1/p-mkemp6/0/salsalehi3/Organoid_Suhail/simulations/sim 64 
2.4.1


In [12]:
# Load necessary models and directories
# CNN model
from keras.applications.vgg16 import VGG16 
CNN_model = VGG16()
CNN_model = Model(inputs = CNN_model.inputs, outputs = CNN_model.layers[-2].output)

# SVM model
pkl_filename = r'/storage/coda1/p-mkemp6/0/salsalehi3/Organoid_Suhail/spatial_classifiers_ovr_red.pkl'
SVM_model = pickle.load(open(pkl_filename, 'rb'))

# indecies of selected features
pkl_filename = r"/storage/coda1/p-mkemp6/0/salsalehi3/Organoid_Suhail/selected_features.pkl"
selected_features = pickle.load(open(pkl_filename, 'rb'))

/storage/home/hcoda1/5/salsalehi3/.conda/envs/organoid/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [13]:
# define PSO particles and search space, fitness function 
class Particle():
    def __init__(self):
        self.position = np.random.uniform(-10, 10,(4,1)) # initialization
        self.pbest_position = self.position
        self.pbest_value = - float('inf')
        self.velocity = np.zeros((4,1))
        self.id = None
        
    def __str__(self):
        return #print("I am at ", self.position, " meu pbest is ", self.pbest_position)
    
    def move(self):
        self.position = self.position + self.velocity
        
class Space():  
    def __init__(self, target, target_error, n_particles, simulation_time,CNN_model,SVM_model, selected_features,formula):
        self.rn_seed = 1 # for reproducipility
        self.sim_id = int(0)
        self.simulation_time = simulation_time
        self.target = target
        self.target_error = target_error
        self.n_particles = n_particles
        self.particles = []
        self.gbest_value = -float('inf')
        self.gbest_position = np.random.uniform(-10, 10,(4,1)) #np.array([])
        self.n_time_steps = int(self.simulation_time/500+1)
        self.traces = []
        self.global_best_sim = 0
        self.CNN_model = CNN_model
        self.SVM_model = SVM_model
        self.selected_features = selected_features
        self.formula = formula

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()
   
    def extract_features(self, image, CNN_model):
        '''
        Extracts features from images using a pretrained CNN 
        
        '''
        # load the image as a 224x224 array
        img = load_img(image, target_size=(224,224))
        # convert from 'PIL.Image.Image' to numpy array
        img = np.array(img) 
        reshaped_img = img.reshape(1,224,224,3) 

        # prepare image for model
        imgx = preprocess_input(reshaped_img)
        # get the feature vector
        features = CNN_model.predict(imgx, use_multiprocessing=True)
        return features
    
    def extract_trace(self, path_to_images, CNN_model,SVM_model, selected_features):
        '''
        Stacks spatial features over time into one temporal signal 
        inputs: 
        path_to_images: path to folder containing images with time stamp
        CNN_model: pretrained CNN model to extract features
        SVM_model: pretrained SVM model for spatial classification 
        selected_features: indecies of selected features (created using logistic regression)
        '''
        os.chdir(path_to_images)
        images = []
        while not images:
            with os.scandir(path_to_images) as files:
              # loops through each file in the directory
                for file in files:
                    if file.name.endswith('.png'): #or file.name.endswith('.jpg'):
                      # adds only the image files to the images list
                        images.append(file.name)

        dic_img = {}
        p = r"dump"
        labels = []
        # loop through each image in the dataset
        only_class_labels = []
        for index, image in enumerate (sorted(images)):
            if index>=45:
                break
            # try to extract the features and update the dictionary
            image_name= os.path.splitext(image)[0]
        for ii in range(10): 
            try:
                feat = self.extract_features(image,CNN_model)
                feat = feat[:,selected_features]
                label = SVM_model.predict(feat.reshape(1,-1))
                arr = SVM_model.decision_function(feat.reshape(1,-1)).reshape(1,-1)

                if index ==0 :
                    traj_dist = arr
                else:
                    print("index > 0")
                    traj_dist = np.concatenate((traj_dist, arr), axis = 0)

                labels.append(int(label[0]))
                break

            except Exception as e: 
                print(e)
                print('failed')
                with open(p,'wb') as file:
                    pickle.dump(images,file)
                    
        # get a list of the filenames
        filenames = np.array(list(dic_img.keys()))
        labels =np.array(labels)
        return traj_dist, labels
    
    def convert_imgs_to_traces(self, paths_to_simulations):
        first_time = True
        for path_to_images in paths_to_simulations:
            spatio_temp, labels = self.extract_trace(path_to_images, self.CNN_model,self.SVM_model, self.selected_features)
            spatio_temp= np.expand_dims(spatio_temp, axis=0)
            if first_time:
                self.traces = spatio_temp
                self.traces_labels = labels
                first_time = False
            else:
                self.traces = np.concatenate((self.traces, spatio_temp), axis = 0)
                self.traces_labels = np.concatenate((self.traces_labels, labels), axis = 0)
            
    def fitness(self, particle_id): # compute robustness
        spatio_temp = self.traces[particle_id]
        # Calculating STL robustness
        # spatio_temp has time in axis =0 and signal in axis =1
        x1= torch.tensor(spatio_temp[:,0].reshape([1, -1, 1]), requires_grad=False).float()
        x2= torch.tensor(spatio_temp[:,1].reshape([1, -1, 1]), requires_grad=False).float()
#         x3= torch.tensor(spatio_temp[:,2].reshape([1, -1, 1]), requires_grad=False)
#         x4= torch.tensor(spatio_temp[:,3].reshape([1, -1, 1]), requires_grad=False)
#         x5= torch.tensor(spatio_temp[:,4].reshape([1, -1, 1]), requires_grad=False)
        signal = (x1,x2)
        
        #calculate robustness
        robustness = formula.robustness(signal, scale=0.5)
        return robustness.item()    
                
    def create_simulation_folder(self,sim_id):
        return r'/storage/coda1/p-mkemp6/0/salsalehi3/Organoid_Suhail/simulations/sim %s ' % int(sim_id)
        
    def set_pbest(self):
        # Serial processing
        for ind, particle in enumerate(self.particles):
            particle.id = int(self.sim_id%self.n_particles)
            row = np.append(particle.position,np.array(self.sim_id).astype(int).reshape(1,1), axis=0)
            if ind == 0:
                data = row
            else:
                data = np.append(data,row, axis =1)
            self.sim_id += 1
        data = data.transpose()
        paths_to_simulations = [self.create_simulation_folder(x[4]) for x in data]
        path_to_run_sim = r"/storage/coda1/p-mkemp6/0/salsalehi3/Organoid_Suhail/"
        os.chdir(path_to_run_sim)
        myfile = "test%s.jobs" %self.sim_id
        if not os.path.exists(myfile):
            f= open(myfile,"x")
        f= open(myfile,"w")
        for x in data:
            x= np.round(x,2)
            run_file = 'python MorpheusSetup.py \" %s " " %s " " %s " " %s " " %s " " %s " " %s "\r\n' % ( \
                str(x[0]),str(x[1]),str(x[2]),str(x[3]),\
                str(self.rn_seed),str(int(x[4])),str(self.simulation_time))
            f.write(run_file)
            os.system(run_file) # run in comand line (NOT parallel jobs)
        f.close()
#         print('finished writing to file')
#         terminal_command = "pace-gnu-job -G " +myfile+" -q embers -A GT-mkemp6"
#         print("Terminal command is : ", terminal_command )
#         os.system(terminal_command)
        for sim_folder in paths_to_simulations:
            while not os.path.isdir(sim_folder):
                print('taking a nap!',sim_folder)
                time.sleep(5)
            time.sleep(5)
#         time.sleep(5)
        self.convert_imgs_to_traces(paths_to_simulations)
        for particle in self.particles:
            fitness_cadidate = self.fitness(particle.id)
            if(particle.pbest_value < fitness_cadidate):
                particle.pbest_value = fitness_cadidate
                particle.pbest_position = particle.position
        return fitness_cadidate

    def set_gbest(self):
        for particle in self.particles:
            best_fitness_cadidate = self.fitness(particle.id)
            if(self.gbest_value < best_fitness_cadidate):
                self.gbest_value = best_fitness_cadidate
                self.gbest_position = particle.position
                self.global_best_sim = particle.id
        return self.gbest_value, self.global_best_sim

    def move_particles(self):
        w = 0.5
        c1 = 0.5
        c2 = 0.9
        for particle in self.particles:
            new_velocity = (w*particle.velocity) + (c1*random.random()) * (particle.pbest_position - particle.position) + \
                            (random.random()*c2) * (self.gbest_position - particle.position)
         
            particle.velocity = new_velocity
            particle.move()

In [14]:
## Write the STL formula
trace_len = 1
x = torch.tensor(np.random.rand(trace_len,1) ).float() 
y = torch.tensor(np.random.rand(trace_len,1) ).float() 
# define the stl formula 
a = torch.as_tensor(0.48).float()
b = torch.as_tensor(0.52).float()
x_exp = Expression('x_exp',x)
y_exp = Expression('y_exp',y)
ϕ1 = x_exp >a
ϕ2 = y_exp <b
formula = stlcg.stlcg.Always(subformula=ϕ1& ϕ2, interval=[2,3])
print('formula is :' ,formula)
#visualize the formula
# viz.make_stl_graph(formula)


formula is : ◻ [2, 3]( (x_exp >= input) ∧ (y_exp <= input) )


In [15]:
##  Main Optimization
# set optimization parameters
n_iterations = 20    # int(input("Inform the number of iterations: "))
target_error = 0.1   # float(input("Inform the target error: "))
n_particles = 5      # int(input("Inform the number of particles: "))
target = 1           # 
simulation_time = 500*(trace_len-1)
search_space = Space(target, target_error, n_particles, simulation_time,CNN_model,SVM_model, selected_features,formula)
particles_vector = [Particle() for _ in range(search_space.n_particles)]
search_space.particles = particles_vector
search_space.print_particles()

#main optimization loop
iteration = 0
start_time = time.time()
while(True):
    iter_time = time.time()
    search_space.set_pbest()    
    current_best_rob, sim_id = search_space.set_gbest()
    if(abs(search_space.gbest_value - search_space.target) <= search_space.target_error):
        break
    print("Iteration #",iteration,"run time = " ,(time.time() - iter_time),",global best solution: ",current_best_rob,"from sim",sim_id)
    search_space.move_particles()
    if iteration < n_iterations:
        iteration += 1
    else: 
        break 

print("Total time %s seconds ---" % (time.time() - start_time))
print("The best solution is: ", search_space.gbest_position, " in n_iterations: ", iteration)

/bin/sh: line 1: 177265 Segmentation fault      singularity exec /usr/local/pace-apps/manual/packages/singularity_images/morpheus-2.2.0.sif morpheus -file morpheus_EP_2020_nov.xml > model_output.txt 2>&1
sh: line 1: 177319 Segmentation fault      singularity exec /usr/local/pace-apps/manual/packages/singularity_images/morpheus-2.2.0.sif morpheus -file morpheus_EP_2020_nov.xml > model_output.txt 2>&1


simulations/sim 0 
Finished simulation successfully
--- 2.347062826156616 seconds ---
simulations/sim 1 
Finished simulation successfully
--- 3.0929880142211914 seconds ---
simulations/sim 2 
Finished simulation successfully
--- 3.1302084922790527 seconds ---


sh: line 1: 177636 Segmentation fault      singularity exec /usr/local/pace-apps/manual/packages/singularity_images/morpheus-2.2.0.sif morpheus -file morpheus_EP_2020_nov.xml > model_output.txt 2>&1


simulations/sim 3 
Finished simulation successfully
--- 2.1803207397460938 seconds ---
simulations/sim 4 
Finished simulation successfully
--- 3.1163530349731445 seconds ---
Iteration # 0 run time =  55.45506000518799 ,global best solution:  -3.431396484375 from sim 0
simulations/sim 5 
Finished simulation successfully
--- 3.1100661754608154 seconds ---
simulations/sim 6 
Finished simulation successfully
--- 3.132307767868042 seconds ---
simulations/sim 7 
Finished simulation successfully
--- 3.11897349357605 seconds ---


sh: line 1: 178240 Segmentation fault      singularity exec /usr/local/pace-apps/manual/packages/singularity_images/morpheus-2.2.0.sif morpheus -file morpheus_EP_2020_nov.xml > model_output.txt 2>&1


...

/bin/sh: line 1: 186225 Segmentation fault      singularity exec /usr/local/pace-apps/manual/packages/singularity_images/morpheus-2.2.0.sif morpheus -file morpheus_EP_2020_nov.xml > model_output.txt 2>&1


simulations/sim 71 
Finished simulation successfully
--- 2.689544200897217 seconds ---
simulations/sim 72 
Finished simulation successfully
--- 3.121460199356079 seconds ---
simulations/sim 73 
Finished simulation successfully
--- 3.1095690727233887 seconds ---


/bin/sh: line 1: 186540 Segmentation fault      singularity exec /usr/local/pace-apps/manual/packages/singularity_images/morpheus-2.2.0.sif morpheus -file morpheus_EP_2020_nov.xml > model_output.txt 2>&1


simulations/sim 74 
Finished simulation successfully
--- 2.6732277870178223 seconds ---
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed
cannot identify image file <_io.BytesIO object at 0x2aab9a2fc130>
failed


UnboundLocalError: local variable 'traj_dist' referenced before assignment